In [1]:
! pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 11.3 MB/s eta 0:00:00


In [2]:
! pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [8]:
! pip install langchain-chroma

In [5]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

Enter API key for Groq: ··········


In [9]:
import warnings
warnings.filterwarnings("ignore")

In [31]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder

In [11]:
model = ChatGroq(model="llama3-8b-8192")

In [12]:
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_time': 0.021666667, 'prompt_time': 0.003336887, 'queue_time': 0.090254258, 'total_time': 0.025003554}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-b5d0c000-64f8-4166-9dc6-2fa04bf0e37c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41})

In [13]:
parser = StrOutputParser()

In [14]:
parser.invoke(model.invoke("hi"))

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [ ]:
#from langchain_core.chat_history import BaseChatMessageHistory

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [16]:
input_text = "The meaning of life is 42"
vector = hf.embed_query(input_text)
print(vector[:3])

[0.03531579300761223, -0.004496623296290636, 0.006513927131891251]


In [18]:
loader = TextLoader("/content/chatbot_data.txt")
doc =loader.load()

In [19]:
doc

[Document(metadata={'source': '/content/chatbot_data.txt'}, page_content='\n\n**1. General Information**\n- Our company operates from Monday to Friday, 9 AM to 6 PM.\n- Customers can reset their passwords using the ‘Forgot Password’ option on the login page.\n- Our services are available globally, with multilingual support.\n- We offer personalized recommendations based on customer preferences.\n\n**2. E-commerce Services**\n- Products can be returned within 30 days with the original receipt.\n- International shipping is available to over 50 countries with additional charges.\n- Order tracking is available through the website under the ‘My Orders’ section.\n- Bulk purchasing options are available with special discounts.\n- Payment options include credit/debit cards, PayPal, and cryptocurrency.\n- Automated restocking alerts can notify customers about product availability.\n- Gift wrapping services and personalized notes are available for orders.\n\n**3. Healthcare Services**\n- Custome

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

In [21]:
vectorstore = Chroma.from_documents(documents=splits, embedding=hf)
retriever = vectorstore.as_retriever()

In [22]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x794968d66690>, search_kwargs={})

In [23]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [24]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [25]:
question_answering_chain=create_stuff_documents_chain(model, chat_prompt)

In [26]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [27]:
response = rag_chain.invoke({"input":"How can I track my order?"})

In [28]:
response["answer"]

"You can track your order through the website under the 'My Orders' section."

In [29]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [32]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [33]:
history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)

In [34]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [35]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

In [36]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [37]:
from langchain_core.messages import HumanMessage, AIMessage

In [38]:
chat_history = []

In [39]:
question1 = "What payment methods do you accept?"

In [40]:
message1= rag_chain.invoke({"input": question1, "chat_history": chat_history})

In [41]:
message1["answer"]

'According to the context, we accept payment options including credit/debit cards, PayPal, and cryptocurrency.'

In [42]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)

In [43]:
chat_history

[HumanMessage(content='What payment methods do you accept?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the context, we accept payment options including credit/debit cards, PayPal, and cryptocurrency.', additional_kwargs={}, response_metadata={})]

In [44]:
second_question = "Can I pay using PayPal?"
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(message2["answer"])

Yes, we accept PayPal as one of our payment options.


In [45]:
store = {}

In [46]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [47]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [48]:
conversational_rag_chain.invoke(
    {"input": "What payment methods do you accept?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'According to the context, your company accepts payment through credit/debit cards, PayPal, and cryptocurrency.'

In [49]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What payment methods do you accept?', additional_kwargs={}, response_metadata={}), AIMessage(content='According to the context, your company accepts payment through credit/debit cards, PayPal, and cryptocurrency.', additional_kwargs={}, response_metadata={})])}

In [50]:
conversational_rag_chain.invoke(
    {"input": "Can I pay using PayPal?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Yes, you can pay using PayPal.'

In [51]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What payment methods do you accept?

AI: According to the context, your company accepts payment through credit/debit cards, PayPal, and cryptocurrency.

User: Can I pay using PayPal?

AI: Yes, you can pay using PayPal.



In [52]:
conversational_rag_chain.invoke(
    {"input": "Do you support cryptocurrency payments?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Yes, your company accepts cryptocurrency payments.'

In [53]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What payment methods do you accept?', additional_kwargs={}, response_metadata={}), AIMessage(content='According to the context, your company accepts payment through credit/debit cards, PayPal, and cryptocurrency.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can I pay using PayPal?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, you can pay using PayPal.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Do you support cryptocurrency payments?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, your company accepts cryptocurrency payments.', additional_kwargs={}, response_metadata={})])}

In [54]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What payment methods do you accept?

AI: According to the context, your company accepts payment through credit/debit cards, PayPal, and cryptocurrency.

User: Can I pay using PayPal?

AI: Yes, you can pay using PayPal.

User: Do you support cryptocurrency payments?

AI: Yes, your company accepts cryptocurrency payments.

